# Proyecto del Día 15 - Optimizar Modelos de Machine Learning

El proyecto del día de hoy consiste en tomar un código que ya está desarrollado con Scikit-Learn, y modificarlo agregando varios apspectos que hemos aprendido el día de hoy.

Este programa es un simple análisis usando el modelo de **Bosques Aleatorios** en un dataset de **Scikit-Learn** sobre registros médicos de **pacientes con diabetes**.

### Consigna

Tu trabajo aquí simplemente consiste en ejecutar este código, y analizarlo para identificar qué hace cada línea, y cuál es su función dentro del código. Una vez que te hayas asegurado de haber identificado sus elementos y de haber comprendido su funcionamiento en general, realiza todas las modificaciones que creas necesarias, para que este código incluya:

+ Preprocesamiento de datos con StandardScaler
+ Selección de mejores categorías (investiga cuál es la mejor función de cálculo)
+ Pipelines
+ Evaluación de modelo

Suena fácil, pero te aseguro que poner todo junto en un mismo proyecto, puede hacer que las cosas se compliquen un poco.

Te deseo lo mejor como siempre, esperando que este proyecto implique un buen tiempo de diversión y resolución de problemas.

In [17]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Cargar el dataset de diabetes
diabetes = datasets.load_diabetes()

# Convertir a DataFrame para facilitar el análisis exploratorio
diabetes_df = pd.DataFrame(data=np.c_[diabetes['data'], diabetes['target']],
                           columns=diabetes['feature_names'] + ['target'])

# Convertir 'target' en categorías para clasificación
diabetes_df['target'] = (diabetes_df['target'] > diabetes_df['target'].median()).astype(int)

# División de datos en conjuntos de entrenamiento y prueba
X = diabetes_df.drop('target', axis=1)
y = diabetes_df['target']
X_entrena, X_prueba, y_entrena, y_prueba = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo RandomForest
modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X_entrena, y_entrena)

# Realizar predicciones con el conjunto de prueba
predicciones = modelo.predict(X_prueba)

# Evaluación del modelo
puntaje = modelo.score(X_prueba, y_prueba)
print(f"\nPrecisión del modelo: {puntaje:.2f}")



Precisión del modelo: 0.72


### Código optimizado

Primero que todo, vamos a ir separando cada parte del código en su propia celda. Posteriormente, iremos modificando cada celda (inc. agregar o quitar algunas), según los requerimientos solicitados en la consigna, por lo que, aviso, el despliegue de los cambios aquí NO será lineal.

In [30]:
import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

# Type hints
from sklearn.utils._bunch import Bunch

In [31]:
# Cargar el dataset de diabetes
diabetes: tuple[Bunch, tuple] = datasets.load_diabetes()
type(diabetes)

sklearn.utils._bunch.Bunch

In [32]:
# Convertir a DataFrame para facilitar el análisis exploratorio
diabetes_df: pd.DataFrame = pd.DataFrame(
    data=np.c_[diabetes['data'], diabetes['target']],  # type: ignore
    columns=diabetes['feature_names'] + ['target']     # type: ignore
)
diabetes_df

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0
...,...,...,...,...,...,...,...,...,...,...,...
437,0.041708,0.050680,0.019662,0.059744,-0.005697,-0.002566,-0.028674,-0.002592,0.031193,0.007207,178.0
438,-0.005515,0.050680,-0.015906,-0.067642,0.049341,0.079165,-0.028674,0.034309,-0.018114,0.044485,104.0
439,0.041708,0.050680,-0.015906,0.017293,-0.037344,-0.013840,-0.024993,-0.011080,-0.046883,0.015491,132.0
440,-0.045472,-0.044642,0.039062,0.001215,0.016318,0.015283,-0.028674,0.026560,0.044529,-0.025930,220.0


In [33]:
# Obtener información del dataset
diabetes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     442 non-null    float64
 1   sex     442 non-null    float64
 2   bmi     442 non-null    float64
 3   bp      442 non-null    float64
 4   s1      442 non-null    float64
 5   s2      442 non-null    float64
 6   s3      442 non-null    float64
 7   s4      442 non-null    float64
 8   s5      442 non-null    float64
 9   s6      442 non-null    float64
 10  target  442 non-null    float64
dtypes: float64(11)
memory usage: 38.1 KB


In [34]:
# Convertir 'target' en categorías para clasificación
diabetes_df['target'] = (
    diabetes_df['target'] > diabetes_df['target'].median()
).astype(int)
diabetes_df['target']

0      1
1      0
2      1
3      1
4      0
      ..
437    1
438    0
439    0
440    1
441    0
Name: target, Length: 442, dtype: int32

In [35]:
# Conocer valores únicos de la columna 'target' posterior a los cambios
diabetes_df['target'].unique()

array([1, 0])

In [36]:
# División de datos en conjuntos de entrenamiento y prueba
X = diabetes_df.drop('target', axis=1)
y = diabetes_df['target']

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((353, 10), (89, 10), (353,), (89,))

Nota sobre la implementación de selector de mejores categorías con `SelectKBest`.

Como decía Federico, `chi2` no parece ser la forma de clasificación más adecuada, lo que aparentemente es consistente con la [documentación](https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection). De acuerdo con ella, hay 3 opciones:

+ `chi2`
+ `f_classif`
+ `mutual_info_classif`

También indica que:

> The methods based on F-test estimate the degree of linear dependency between two random variables. On the other hand, mutual information methods can capture any kind of statistical dependency, but being nonparametric, they require more samples for accurate estimation. Note that the `chi2`-test should only be applied to non-negative features, such as frequencies.

In [37]:
pipeline: Pipeline = make_pipeline(
    StandardScaler(),
    SelectKBest(f_classif, k=5),
    RandomForestClassifier(n_estimators=100, random_state=42)
)

In [38]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('selectkbest', SelectKBest(k=5)),
                ('randomforestclassifier',
                 RandomForestClassifier(random_state=42))])

In [39]:
# Realizar predicciones con el conjunto de prueba
y_pred: np.ndarray | tuple = pipeline.predict(X_test)
y_pred

array([0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1])

In [40]:
# Evaluación del modelo
score: float = float(pipeline.score(X_test, y_test))
print(f"\nPrecisión del modelo: {score:.2f}")


Precisión del modelo: 0.73
